In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from pathlib import Path
import cv2
from glob import glob
from PIL import Image
from keras import layers
from keras import models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold

# 데이콘 이상치 탐색

**https://dacon.io/competitions/official/235894/overview/description**

In [4]:
path = "C:/Users/choij/Desktop/datasets/open/open/train/train/"
#train dataset 위치

In [5]:
path2 = 'C:/Users/choij/Desktop/datasets/open/open/test/test/'
#test 데이터셋 위치

## 데이터 전처리

In [8]:
df = pd.read_csv("C:/Users/choij/Desktop/datasets/open/open/train_df.csv")
df #1024 * 1024 * 3 형식의 이미지들

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good
...,...,...,...,...,...
4272,4272,14272.png,transistor,good,transistor-good
4273,4273,14273.png,transistor,good,transistor-good
4274,4274,14274.png,grid,good,grid-good
4275,4275,14275.png,zipper,good,zipper-good


In [9]:
print(len(df['state'].value_counts())) 
print(len(df['class'].value_counts())) 
print(len(df['label'].value_counts())) # 88개의 클래스 분류

49
15
88


In [10]:
df['label'].value_counts()

hazelnut-good              391
screw-good                 320
carpet-good                280
pill-good                  267
grid-good                  264
                          ... 
cable-poke_insulation        5
pill-pill_type               5
transistor-damaged_case      5
transistor-bent_lead         5
wood-color                   4
Name: label, Length: 88, dtype: int64

In [11]:
df = df.drop(['class','state' , 'file_name'], axis = 1)

In [12]:
df.head()

,index,label
0,0,transistor-good
1,1,capsule-good
2,2,transistor-good
3,3,wood-good
4,4,bottle-good


In [13]:
categories = np.unique(df['label'])
categories #클래스 집합 리스트

array(['bottle-broken_large', 'bottle-broken_small',
       'bottle-contamination', 'bottle-good', 'cable-bent_wire',
       'cable-cable_swap', 'cable-combined', 'cable-cut_inner_insulation',
       'cable-cut_outer_insulation', 'cable-good', 'cable-missing_cable',
       'cable-missing_wire', 'cable-poke_insulation', 'capsule-crack',
       'capsule-faulty_imprint', 'capsule-good', 'capsule-poke',
       'capsule-scratch', 'capsule-squeeze', 'carpet-color', 'carpet-cut',
       'carpet-good', 'carpet-hole', 'carpet-metal_contamination',
       'carpet-thread', 'grid-bent', 'grid-broken', 'grid-glue',
       'grid-good', 'grid-metal_contamination', 'grid-thread',
       'hazelnut-crack', 'hazelnut-cut', 'hazelnut-good', 'hazelnut-hole',
       'hazelnut-print', 'leather-color', 'leather-cut', 'leather-fold',
       'leather-glue', 'leather-good', 'leather-poke', 'metal_nut-bent',
       'metal_nut-color', 'metal_nut-flip', 'metal_nut-good',
       'metal_nut-scratch', 'pill-color', 'p

In [14]:
nb_classes = len(categories)
nb_classes
#https://velog.io/@jaehyeong/CNN-%EB%AA%A8%EB%8D%B8%EC%9D%84-%ED%86%B5%ED%95%9C-%EC%9E%90%EB%8F%99%EC%B0%A8-%EC%82%AC%EA%B3%A0-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EB%B6%84%EB%A5%98

88

In [15]:
base_dir = "C:/Users/choij/Desktop/datasets/open/open/train/small_train"

In [16]:
test_base_dir = "C:/Users/choij/Desktop/datasets/open/open/test/small_test"

In [79]:
os.mkdir(base_dir)

In [121]:
os.mkdir(test_base_dir)

In [17]:
fname_list = os.listdir(base_dir)

In [18]:
base = base_dir + '/'
base

'C:/Users/choij/Desktop/datasets/open/open/train/small_train/'

In [19]:
test_base = test_base_dir + '/'

In [80]:
for fname in categories:
    fnames = os.path.join(base_dir, fname)
    os.mkdir(fnames) # class 별로 폴더 생성

In [124]:
for fname in categories:
    fnames = os.path.join(test_base_dir, fname)
    os.mkdir(fnames)

In [20]:
format = [".jpg",".png",".jpeg","bmp",".JPG",".PNG","JPEG","BMP"]

In [29]:
for (path, dirs, files) in os.walk(path):
    for file in files:
        if file.endswith(tuple(format)):
            image = Image.open(path+"/"+file)
            print(image.filename)
            print(image.size)
            image=image.resize((int(image.size[0]*0.2), int(image.size[1]*0.2)))
            image.save(path+"/"+file)
            print(image.size)

        else:
            print(path)
            print("InValid",file) # 훈련 파일크기 작게

C:/Users/choij/Desktop/datasets/open/open/train/train/10000.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10001.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/train/train/10002.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10003.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10004.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/train/train/10005.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10006.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/train/train/10007.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10008.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10009.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10010.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/op

(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10097.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10098.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10099.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/10100.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/train/train/10101.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10102.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10103.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/10104.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/train/train/10105.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10106.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/10107.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datase

(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10193.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10194.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10195.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10196.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10197.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10198.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/10199.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10200.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10201.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10202.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10203.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/

C:/Users/choij/Desktop/datasets/open/open/train/train/10290.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10291.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10292.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10293.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10294.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/10295.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10296.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10297.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10298.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/train/train/10299.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/train/train/10300.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open

(180, 180)
C:/Users/choij/Desktop/datasets/open/open/train/train/10386.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10387.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10388.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/10389.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10390.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/train/train/10391.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10392.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10393.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/train/train/10394.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/train/train/10395.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/10396.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datase

...

C:/Users/choij/Desktop/datasets/open/open/train/train/13826.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13827.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13828.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13829.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13830.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/13831.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/13832.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13833.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13834.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/train/train/13835.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/train/train/13836.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/ope

C:/Users/choij/Desktop/datasets/open/open/train/train/13922.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13923.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13924.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/13925.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/train/train/13926.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13927.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13928.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13929.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13930.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/13931.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/train/train/13932.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/o

(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14018.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14019.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/train/train/14020.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/train/train/14021.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14022.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14023.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14024.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14025.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14026.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14027.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/train/train/14028.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/data

(160, 160)
C:/Users/choij/Desktop/datasets/open/open/train/train/14114.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/14115.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14116.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/14117.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14118.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14119.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14120.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14121.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14122.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14123.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14124.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/data

C:/Users/choij/Desktop/datasets/open/open/train/train/14208.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14209.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14210.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14211.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14212.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14213.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14214.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14215.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/train/train/14216.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/train/train/14217.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/train/train/14218.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/op

In [128]:
for (path, dirs, files) in os.walk(path2):
    for file in files:
        if file.endswith(tuple(format)):
            image = Image.open(path+"/"+file)
            print(image.filename)
            print(image.size)
            image=image.resize((int(image.size[0]*0.2), int(image.size[1]*0.2)))
            image.save(path+"/"+file)
            print(image.size)

        else:
            print(path)
            print("InValid",file) # 테스트 파일크기 작게

C:/Users/choij/Desktop/datasets/open/open/test/test//20000.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20001.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20002.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20003.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20004.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20005.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//20006.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//20007.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20008.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20009.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20010.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/test/test//

(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20096.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20097.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20098.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//20099.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//20100.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20101.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20102.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//20103.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//20104.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20105.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/test/test//20106.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/ope

(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//20192.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20193.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20194.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//20195.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20196.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20197.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20198.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20199.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20200.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20201.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20202.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/op

C:/Users/choij/Desktop/datasets/open/open/test/test//20287.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20288.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20289.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/test/test//20290.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20291.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/test/test//20292.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20293.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20294.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20295.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20296.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20297.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//

(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20382.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20383.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//20384.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//20385.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20386.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//20387.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//20388.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20389.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20390.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//20391.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//20392.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/ope

...

(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21734.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21735.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21736.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//21737.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21738.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21739.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21740.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21741.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21742.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//21743.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//21744.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open

C:/Users/choij/Desktop/datasets/open/open/test/test//21829.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//21830.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21831.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21832.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//21833.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21834.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21835.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21836.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//21837.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21838.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21839.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/te

C:/Users/choij/Desktop/datasets/open/open/test/test//21926.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21927.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//21928.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/test/test//21929.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21930.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21931.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//21932.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21933.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21934.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/test/test//21935.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//21936.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//

C:/Users/choij/Desktop/datasets/open/open/test/test//22023.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22024.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//22025.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22026.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//22027.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//22028.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22029.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22030.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22031.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22032.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22033.png
(700, 700)
(140, 140)
C:/Users/choij/Desktop/datasets/open/open/tes

(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22118.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22119.png
(800, 800)
(160, 160)
C:/Users/choij/Desktop/datasets/open/open/test/test//22120.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22121.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22122.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22123.png
(840, 840)
(168, 168)
C:/Users/choij/Desktop/datasets/open/open/test/test//22124.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22125.png
(1000, 1000)
(200, 200)
C:/Users/choij/Desktop/datasets/open/open/test/test//22126.png
(900, 900)
(180, 180)
C:/Users/choij/Desktop/datasets/open/open/test/test//22127.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/open/test/test//22128.png
(1024, 1024)
(204, 204)
C:/Users/choij/Desktop/datasets/open/o

In [84]:
for idx in range(len(os.listdir(path))): #0~4277 반복하면서 훈련 이미지 복사
    index = 10000
    for fname in fname_list: #폴더 리스트 
        if df['label'][idx] == fname:
            index += idx
            index = str(index) + '.png' 
            path_dir = base_dir + '/'+ fname
            shutil.copyfile(path+index, path_dir + '/' + str(index) + '.png')

In [132]:
for idx in range(len(os.listdir(path2))): #0~4277 반복 테스트 이미지 복사
    index = 20000
    for fname in fname_list: #폴더 리스트 1개씩 흘음 
        if df['label'][idx] == fname:
            index += idx
            index = str(index) + '.png' 
            path_dir = test_base_dir + '/'+ fname
            shutil.copyfile(path2+index, path_dir + '/' + str(index))

In [22]:
def CustomModel():
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5,5), activation = 'relu', input_shape = (150,150,3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(32, (5,5), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(64, (5,5), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(64, (1,1), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(128, (5,5), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation = 'relu'))
    model.add(layers.Dense(88, activation = 'softmax'))
    return model

In [164]:
glob(base_dir + '/'+ 'transistor-good'+ '/*.png')

['C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10000.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10002.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10009.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10042.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10049.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10072.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10091.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10097.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10135.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/small_train/transistor-good\\10138.png.png',
 'C:/Users/choij/Desktop/datasets/open/open/train/

In [23]:
image_w = 150
image_h = 150

X = []
Y = []

for idx, cat in enumerate(categories):
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    files = glob(base_dir + '/' + cat + '/*.png')
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert('RGB')
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)
        if i % 10 ==0:
            print(i, '\n', data)


0 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
0 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 2

100 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
110 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 2

0 
 [[[ 84 114 138]
  [ 84 115 137]
  [ 82 111 132]
  ...
  [ 69  93 107]
  [ 61  80  91]
  [ 53  66  75]]

 [[ 84 116 141]
  [ 87 119 141]
  [ 82 110 129]
  ...
  [ 65  83  95]
  [ 58  72  82]
  [ 52  63  71]]

 [[ 82 116 143]
  [ 87 120 144]
  [ 80 107 126]
  ...
  [ 55  68  78]
  [ 49  59  65]
  [ 46  55  60]]

 ...

 [[ 72  95 114]
  [ 71  96 117]
  [ 69  94 115]
  ...
  [ 66  89 106]
  [ 64  85 103]
  [ 64  84 102]]

 [[ 75  99 118]
  [ 72  95 115]
  [ 67  89 106]
  ...
  [ 67  89 106]
  [ 63  84 101]
  [ 58  77  93]]

 [[ 75 100 118]
  [ 74  99 117]
  [ 72  97 114]
  ...
  [ 71  95 115]
  [ 66  87 106]
  [ 55  73  87]]]
0 
 [[[ 78 102 123]
  [ 81 108 135]
  [ 75 100 124]
  ...
  [ 61  78  97]
  [ 54  65  79]
  [ 47  55  67]]

 [[ 81 108 131]
  [ 81 111 140]
  [ 72  98 124]
  ...
  [ 62  80 101]
  [ 50  61  71]
  [ 43  50  59]]

 [[ 82 109 133]
  [ 80 108 138]
  [ 67  90 115]
  ...
  [ 60  75  95]
  [ 54  66  79]
  [ 53  65  78]]

 ...

 [[ 55  72  97]
  [ 54  73  98]
  [ 60  80 1

120 
 [[[ 81 110 138]
  [ 76 101 129]
  [ 65  83 106]
  ...
  [ 55  73 100]
  [ 61  81 108]
  [ 65  84 109]]

 [[ 73  95 122]
  [ 64  82 107]
  [ 63  80 100]
  ...
  [ 62  84 108]
  [ 58  77 104]
  [ 58  75  98]]

 [[ 65  81 102]
  [ 53  65  83]
  [ 63  81  98]
  ...
  [ 80 113 141]
  [ 65  89 118]
  [ 59  78 103]]

 ...

 [[ 57  72  92]
  [ 63  79  98]
  [ 78 102 125]
  ...
  [ 55  64  76]
  [ 67  85 105]
  [ 85 116 143]]

 [[ 60  78  98]
  [ 72  95 117]
  [ 84 114 141]
  ...
  [ 61  74  92]
  [ 71  92 115]
  [ 84 114 144]]

 [[ 73  99 124]
  [ 82 112 139]
  [ 82 109 134]
  ...
  [ 66  82 103]
  [ 83 111 141]
  [103 149 187]]]
130 
 [[[ 52  66  79]
  [ 47  58  67]
  [ 42  51  59]
  ...
  [ 63  85 105]
  [ 59  81 100]
  [ 60  82 104]]

 [[ 48  60  70]
  [ 50  63  74]
  [ 49  62  73]
  ...
  [ 67  92 114]
  [ 60  84 105]
  [ 59  83 104]]

 [[ 56  73  86]
  [ 56  70  82]
  [ 52  65  76]
  ...
  [ 70  96 116]
  [ 64  89 108]
  [ 58  80  98]]

 ...

 [[ 47  59  69]
  [ 43  52  61]
  [ 42  

0 
 [[[232 230 228]
  [229 232 228]
  [230 231 227]
  ...
  [224 222 226]
  [225 224 226]
  [226 224 223]]

 [[227 229 227]
  [229 230 227]
  [229 229 229]
  ...
  [224 223 226]
  [225 225 226]
  [224 222 223]]

 [[227 227 227]
  [229 230 227]
  [227 229 228]
  ...
  [226 224 224]
  [227 224 224]
  [225 222 225]]

 ...

 [[225 229 219]
  [225 229 220]
  [221 226 218]
  ...
  [214 214 209]
  [214 213 210]
  [213 211 210]]

 [[225 229 219]
  [223 228 219]
  [224 227 218]
  ...
  [214 214 210]
  [211 214 211]
  [210 211 210]]

 [[224 226 220]
  [223 226 220]
  [225 227 217]
  ...
  [210 211 209]
  [212 212 211]
  [211 212 210]]]
10 
 [[[211 216 217]
  [211 214 218]
  [208 214 219]
  ...
  [208 209 214]
  [209 211 213]
  [209 210 213]]

 [[212 216 218]
  [210 214 216]
  [209 215 217]
  ...
  [208 210 215]
  [207 210 212]
  [207 209 213]]

 [[212 216 218]
  [211 215 218]
  [210 214 216]
  ...
  [209 211 215]
  [206 210 213]
  [206 209 213]]

 ...

 [[206 211 207]
  [207 214 208]
  [206 213 

...

160 
 [[[143  98  87]
  [143  99  86]
  [147 101  88]
  ...
  [149 104  90]
  [146 102  89]
  [143 101  88]]

 [[143  97  86]
  [142  98  85]
  [144  99  87]
  ...
  [150 103  90]
  [147 103  90]
  [146 103  90]]

 [[140  96  85]
  [141  97  84]
  [141  97  84]
  ...
  [150 104  91]
  [149 103  90]
  [150 104  90]]

 ...

 [[142  97  86]
  [142  98  85]
  [138  95  83]
  ...
  [147 102  88]
  [146 100  88]
  [149 103  91]]

 [[140  97  84]
  [140  97  84]
  [138  96  84]
  ...
  [148 103  90]
  [146 101  88]
  [149 102  89]]

 [[136  93  81]
  [139  96  81]
  [140  95  82]
  ...
  [150 103  87]
  [147 101  87]
  [146 101  88]]]
170 
 [[[160 113 102]
  [161 116 103]
  [160 114 101]
  ...
  [155 108  92]
  [155 109  95]
  [157 110  95]]

 [[156 111  99]
  [158 113 100]
  [157 112 100]
  ...
  [157 110  95]
  [158 111  96]
  [158 111  95]]

 [[154 110  97]
  [159 113  99]
  [158 113 100]
  ...
  [158 111  95]
  [159 112  96]
  [158 111  96]]

 ...

 [[143 101  91]
  [143 101  91]
  [143 1

60 
 [[[178 138 105]
  [170 126  93]
  [168 122  90]
  ...
  [143  98  73]
  [134  86  65]
  [142 100  76]]

 [[182 141 106]
  [174 129  94]
  [168 122  89]
  ...
  [145  99  75]
  [137  89  66]
  [144 101  77]]

 [[184 141 107]
  [180 133  98]
  [174 126  93]
  ...
  [145  99  75]
  [137  89  66]
  [147 103  78]]

 ...

 [[188 152 109]
  [181 140  99]
  [175 137  98]
  ...
  [149 108  79]
  [139 102  75]
  [129  94  71]]

 [[183 147 106]
  [176 137  96]
  [173 137  99]
  ...
  [144 104  76]
  [141 104  76]
  [126  92  69]]

 [[174 137  98]
  [170 131  92]
  [170 132  94]
  ...
  [140 102  75]
  [141 102  76]
  [123  91  69]]]
70 
 [[[188 147 115]
  [195 158 123]
  [176 133 102]
  ...
  [127  91  70]
  [142 101  76]
  [128  90  69]]

 [[190 149 115]
  [197 158 122]
  [172 127  96]
  ...
  [132  94  73]
  [146 104  79]
  [131  92  70]]

 [[196 155 119]
  [202 161 126]
  [171 125  94]
  ...
  [134  96  72]
  [149 106  80]
  [130  92  70]]

 ...

 [[192 150 106]
  [198 158 113]
  [202 167

210 
 [[[185 133 101]
  [175 123  94]
  [175 124  95]
  ...
  [148 104  80]
  [153 104  80]
  [164 116  88]]

 [[188 136 103]
  [180 129  98]
  [177 127  97]
  ...
  [149 103  79]
  [151 102  76]
  [168 119  90]]

 [[187 135 104]
  [186 133 102]
  [185 133 101]
  ...
  [139  92  70]
  [152 101  76]
  [170 121  91]]

 ...

 [[172 125  89]
  [172 129  94]
  [185 140 102]
  ...
  [179 131  97]
  [185 138 103]
  [191 144 109]]

 [[173 125  90]
  [170 127  93]
  [184 138 100]
  ...
  [176 127  95]
  [181 136 101]
  [183 139 105]]

 [[166 121  87]
  [169 126  93]
  [184 138 101]
  ...
  [174 127  94]
  [179 133  99]
  [178 136 103]]]
220 
 [[[157 112  84]
  [169 123  92]
  [186 137 104]
  ...
  [163 113  84]
  [148 105  81]
  [129  87  68]]

 [[159 115  86]
  [173 125  94]
  [193 144 108]
  ...
  [168 117  87]
  [153 110  84]
  [128  86  68]]

 [[165 118  88]
  [176 128  96]
  [195 143 107]
  ...
  [171 120  89]
  [157 114  87]
  [134  91  72]]

 ...

 [[188 135  94]
  [188 139  99]
  [192 1

60 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
70 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255

220 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
230 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 2

In [24]:
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

(4277, 150, 150, 3)
(4277, 88)


In [25]:
x_train = X
y_train = Y

In [26]:
image_w = 150
image_h = 150

test_X = []
test_Y = []

for idx, cat in enumerate(categories):
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    files = glob(test_base_dir + '/' + cat + '/*.png')
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert('RGB')
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        test_X.append(data)
        test_Y.append(label)
        if i % 10 ==0:
            print(i, '\n', data)


0 
 [[[15 15 24]
  [15 14 27]
  [15 15 26]
  ...
  [15 15 25]
  [14 14 25]
  [14 14 24]]

 [[15 15 25]
  [15 15 26]
  [15 15 25]
  ...
  [15 15 25]
  [15 14 24]
  [14 14 24]]

 [[15 16 26]
  [16 15 25]
  [16 15 26]
  ...
  [15 15 25]
  [15 16 26]
  [14 14 25]]

 ...

 [[15 15 26]
  [15 15 26]
  [15 15 25]
  ...
  [17 18 26]
  [15 16 25]
  [15 15 25]]

 [[17 17 29]
  [16 16 27]
  [15 15 24]
  ...
  [16 17 26]
  [15 16 25]
  [14 15 25]]

 [[17 17 29]
  [16 16 26]
  [15 15 24]
  ...
  [14 15 26]
  [15 15 25]
  [15 15 25]]]
0 
 [[[12 12 22]
  [12 12 22]
  [13 12 22]
  ...
  [13 13 22]
  [13 12 21]
  [12 12 22]]

 [[13 13 22]
  [13 13 23]
  [13 12 22]
  ...
  [13 12 21]
  [12 12 21]
  [13 12 22]]

 [[13 14 24]
  [13 13 23]
  [13 13 23]
  ...
  [13 13 22]
  [13 12 21]
  [12 12 20]]

 ...

 [[13 12 22]
  [12 12 21]
  [12 12 21]
  ...
  [13 13 22]
  [13 13 22]
  [13 13 22]]

 [[12 12 21]
  [12 12 22]
  [12 12 21]
  ...
  [13 13 21]
  [12 13 22]
  [13 13 23]]

 [[12 12 21]
  [12 12 21]
  [13 12

40 
 [[[ 68  85 106]
  [ 72  91 111]
  [ 74  97 119]
  ...
  [ 75  98 122]
  [ 75  97 121]
  [ 70  87 110]]

 [[ 59  73  88]
  [ 71  90 110]
  [ 74  97 121]
  ...
  [ 76  98 124]
  [ 71  91 114]
  [ 65  79  99]]

 [[ 64  81  98]
  [ 70  89 111]
  [ 71  90 114]
  ...
  [ 71  90 115]
  [ 69  85 107]
  [ 67  84 103]]

 ...

 [[ 62  83 110]
  [ 60  80 108]
  [ 57  77 105]
  ...
  [ 63  80 101]
  [ 54  67  85]
  [ 58  72  88]]

 [[ 72  97 125]
  [ 68  92 118]
  [ 63  85 110]
  ...
  [ 60  76  95]
  [ 54  67  85]
  [ 58  72  89]]

 [[ 66  89 115]
  [ 61  80 102]
  [ 60  79  98]
  ...
  [ 57  71  88]
  [ 54  66  82]
  [ 59  73  91]]]
50 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0

40 
 [[[190 190 190]
  [190 190 190]
  [189 189 189]
  ...
  [191 191 191]
  [193 193 193]
  [191 191 191]]

 [[190 190 190]
  [191 191 191]
  [189 189 189]
  ...
  [190 190 190]
  [190 190 190]
  [190 190 190]]

 [[192 192 192]
  [192 192 192]
  [191 191 191]
  ...
  [190 190 190]
  [192 192 192]
  [192 192 192]]

 ...

 [[179 179 179]
  [179 179 179]
  [180 180 180]
  ...
  [184 184 184]
  [185 185 185]
  [185 185 185]]

 [[180 180 180]
  [180 180 180]
  [180 180 180]
  ...
  [184 184 184]
  [184 184 184]
  [184 184 184]]

 [[179 179 179]
  [178 178 178]
  [180 180 180]
  ...
  [186 186 186]
  [184 184 184]
  [184 184 184]]]
50 
 [[[13 14 23]
  [14 14 24]
  [14 13 24]
  ...
  [14 14 23]
  [13 13 22]
  [13 13 22]]

 [[13 13 23]
  [14 14 23]
  [14 13 23]
  ...
  [15 16 25]
  [14 14 23]
  [12 12 22]]

 [[13 13 23]
  [13 13 23]
  [13 13 24]
  ...
  [13 13 22]
  [13 12 23]
  [13 12 21]]

 ...

 [[13 13 23]
  [15 15 25]
  [16 15 25]
  ...
  [14 15 24]
  [13 13 23]
  [13 13 23]]

 [[13 13 2

30 
 [[[25 26 31]
  [24 26 32]
  [25 25 31]
  ...
  [25 24 30]
  [25 25 31]
  [25 25 31]]

 [[23 24 31]
  [24 25 31]
  [25 25 32]
  ...
  [25 25 31]
  [25 26 31]
  [25 25 31]]

 [[24 24 31]
  [25 25 32]
  [25 25 31]
  ...
  [25 25 31]
  [25 26 31]
  [24 24 30]]

 ...

 [[26 26 33]
  [25 25 33]
  [25 25 32]
  ...
  [25 24 30]
  [24 24 31]
  [26 25 31]]

 [[26 26 33]
  [25 26 33]
  [25 25 32]
  ...
  [24 24 31]
  [24 24 30]
  [25 24 31]]

 [[25 25 32]
  [24 25 30]
  [25 25 31]
  ...
  [23 24 30]
  [25 24 31]
  [25 25 31]]]
40 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
50 
 [[[16 15 28]
  [17 17 27]
  [16 16 26]
  ...
  [1

30 
 [[[108  72  61]
  [110  74  61]
  [141  93  75]
  ...
  [103  67  56]
  [ 95  61  51]
  [ 96  62  52]]

 [[129  85  70]
  [122  80  65]
  [126  82  66]
  ...
  [122  79  64]
  [111  71  59]
  [111  73  60]]

 [[122  79  65]
  [138  91  75]
  [123  82  67]
  ...
  [115  74  60]
  [116  75  62]
  [116  75  61]]

 ...

 [[ 83  56  50]
  [ 86  58  52]
  [ 90  60  52]
  ...
  [103  67  57]
  [ 89  59  51]
  [ 84  54  48]]

 [[ 98  66  56]
  [ 99  68  58]
  [103  70  59]
  ...
  [112  73  62]
  [104  69  59]
  [ 86  56  49]]

 [[ 96  64  56]
  [ 92  63  53]
  [ 92  62  53]
  ...
  [ 97  63  55]
  [106  70  60]
  [101  68  58]]]
40 
 [[[156 107  95]
  [158 108  95]
  [156 107  93]
  ...
  [159 108  95]
  [160 110  97]
  [157 109  95]]

 [[154 106  93]
  [157 107  94]
  [159 108  94]
  ...
  [159 109  95]
  [161 111  97]
  [160 110  95]]

 [[149 102  90]
  [153 105  91]
  [158 107  93]
  ...
  [162 111  96]
  [161 110  96]
  [161 110  96]]

 ...

 [[156 106  94]
  [155 105  93]
  [154 103

...

40 
 [[[16 17 22]
  [17 17 23]
  [16 16 22]
  ...
  [20 19 25]
  [19 18 25]
  [19 19 24]]

 [[16 17 22]
  [17 16 23]
  [15 16 23]
  ...
  [19 18 25]
  [19 17 25]
  [19 18 24]]

 [[17 17 22]
  [16 16 24]
  [16 16 23]
  ...
  [19 18 25]
  [19 18 26]
  [19 18 25]]

 ...

 [[19 18 23]
  [19 18 23]
  [18 18 23]
  ...
  [17 17 23]
  [17 17 23]
  [17 16 22]]

 [[18 18 23]
  [19 18 24]
  [18 18 23]
  ...
  [17 17 23]
  [18 18 23]
  [17 17 22]]

 [[18 18 23]
  [18 18 23]
  [18 18 23]
  ...
  [16 16 21]
  [16 17 22]
  [16 17 22]]]
50 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
60 
 [[[151 155 144]
  [137 140 130]
  [147 150 138]
 

30 
 [[[152 148 140]
  [145 145 137]
  [166 167 156]
  ...
  [ 65  64  60]
  [101 102  93]
  [147 156 142]]

 [[171 170 165]
  [165 164 160]
  [156 154 149]
  ...
  [ 86  83  75]
  [105 105  95]
  [132 137 127]]

 [[136 132 129]
  [140 135 130]
  [118 110 105]
  ...
  [ 93  93  84]
  [ 94  93  84]
  [ 99  97  89]]

 ...

 [[149 143 136]
  [146 142 137]
  [159 161 155]
  ...
  [123 122 112]
  [145 146 135]
  [149 151 140]]

 [[171 166 162]
  [173 170 165]
  [179 181 175]
  ...
  [139 140 129]
  [142 144 132]
  [144 146 137]]

 [[163 159 150]
  [174 173 164]
  [183 184 176]
  ...
  [144 144 134]
  [116 116 105]
  [123 125 114]]]
0 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 

0 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
0 
 [[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [208 208 208]
  [209 209 209]
  [196 196 196]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [206 206 206]
  [207 207 207]
  [208 208 208]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [206 206 206]
  [208 208 208]
  [210 210 210]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ..

0 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
0 
 [[[25 25 31]
  [24 25 30]
  [25 25 31]
  ...
  [23 23 31]
  [24 22 30]
  [23 23 30]]

 [[25 25 31]
  [24 25 32]
  [26 25 31]
  ...
  [23 24 31]
  [23 24 30]
  [23 24 30]]

 [[24 25 32]
  [25 25 31]
  [26 25 31]
  ...
  [23 24 31]
  [23 24 30]
  [25 23 30]]

 ...

 [[24 24 30]
  [25 25 31]
  [24 25 31]
  ...
  [24 24 31]
  [23 24 31]
  [24 24 31]]

 [[25 25 31]
  [25 25 31]
  [25 25 32]
  ...
  [24 24 31]
  [25 24 31]
  [24 25 31]]

 [[25 25 32]
  [26 25 31]
  [27 25 31]
  ...
  [24 24 30]
  [25 24 31]
  [25 24 31]]]
0 
 [[[132 135 125]
  [127 132 120]
  [135 139 129]
  ..

110 
 [[[108  71  60]
  [ 97  63  54]
  [102  66  57]
  ...
  [ 91  58  49]
  [ 98  64  54]
  [ 87  56  48]]

 [[104  69  60]
  [ 96  61  53]
  [111  73  61]
  ...
  [ 93  61  51]
  [ 90  56  48]
  [ 97  62  52]]

 [[123  81  68]
  [103  66  57]
  [ 98  62  55]
  ...
  [112  74  62]
  [ 86  57  50]
  [ 93  58  49]]

 ...

 [[ 85  58  51]
  [102  69  59]
  [103  70  59]
  ...
  [104  67  58]
  [110  72  61]
  [106  69  58]]

 [[ 87  59  51]
  [ 84  56  50]
  [ 88  60  52]
  ...
  [113  73  62]
  [110  72  61]
  [103  67  58]]

 [[107  72  62]
  [ 99  67  58]
  [107  72  61]
  ...
  [104  68  58]
  [ 97  63  54]
  [ 93  60  51]]]
120 
 [[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 ...

 [[41 41 45]
  [42 42 46]
  [42 41 46]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[41 42

In [27]:
test_x = np.array(test_X)
y_test = np.array(test_Y)

print(test_x.shape)
print(y_test.shape)

(2154, 150, 150, 3)
(2154, 88)


In [28]:
x_train = x_train.astype('float') / 255
x_test = test_x.astype('float') / 255

In [29]:
y_train.argmax(1)

array([ 0,  0,  0, ..., 87, 87, 87], dtype=int64)

In [30]:
print(x_train.shape)
print(y_train.shape)

(4277, 150, 150, 3)
(4277, 88)


In [46]:
for i,fname in enumerate(os.listdir(test_base_dir)):
    print('클래스:{}, 이미지 수 :{}'.format(i, len(os.listdir(test_base_dir + '/'+fname))))
        

클래스:0, 이미지 수 :8
클래스:1, 이미지 수 :7
클래스:2, 이미지 수 :6
클래스:3, 이미지 수 :112
클래스:4, 이미지 수 :4
클래스:5, 이미지 수 :3
클래스:6, 이미지 수 :2
클래스:7, 이미지 수 :4
클래스:8, 이미지 수 :3
클래스:9, 이미지 수 :103
클래스:10, 이미지 수 :4
클래스:11, 이미지 수 :2
클래스:12, 이미지 수 :3
클래스:13, 이미지 수 :6
클래스:14, 이미지 수 :6
클래스:15, 이미지 수 :110
클래스:16, 이미지 수 :5
클래스:17, 이미지 수 :5
클래스:18, 이미지 수 :4
클래스:19, 이미지 수 :6
클래스:20, 이미지 수 :4
클래스:21, 이미지 수 :147
클래스:22, 이미지 수 :5
클래스:23, 이미지 수 :7
클래스:24, 이미지 수 :4
클래스:25, 이미지 수 :2
클래스:26, 이미지 수 :3
클래스:27, 이미지 수 :4
클래스:28, 이미지 수 :131
클래스:29, 이미지 수 :2
클래스:30, 이미지 수 :4
클래스:31, 이미지 수 :3
클래스:32, 이미지 수 :5
클래스:33, 이미지 수 :185
클래스:34, 이미지 수 :6
클래스:35, 이미지 수 :4
클래스:36, 이미지 수 :4
클래스:37, 이미지 수 :6
클래스:38, 이미지 수 :2
클래스:39, 이미지 수 :5
클래스:40, 이미지 수 :125
클래스:41, 이미지 수 :4
클래스:42, 이미지 수 :7
클래스:43, 이미지 수 :6
클래스:44, 이미지 수 :5
클래스:45, 이미지 수 :127
클래스:46, 이미지 수 :4
클래스:47, 이미지 수 :8
클래스:48, 이미지 수 :4
클래스:49, 이미지 수 :6
클래스:50, 이미지 수 :6
클래스:51, 이미지 수 :8
클래스:52, 이미지 수 :130
클래스:53, 이미지 수 :4
클래스:54, 이미지 수 :7
클래스:55, 이미지 수 :153
클래스:56, 이미지 수 :8
클래스:57, 이미지 수 :5
클래스:

In [48]:
skf = StratifiedKFold(n_splits = 15, random_state = 32, shuffle = True)
n = 0

cnn_pred = []

for train_index, valid_index in skf.split(x_train, y_train.argmax(1)):
    n += 1
    x_train, x_valid = x_train[train_index], x_train[valid_index]
    y_train, y_valid = y_train[train_index], y_train[valid_index]
    
    model = CustomModel()
    
    model.compile(optimizer = 'Adam', 
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy'])
    
    filename = 'CNN_checkpoint3.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filename,
                                                   monitor = 'val_accuracy',
                                                   verbose = 0,
                                                   save_best_only = True,
                                                   mode = 'auto')

    earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                    patience = 6)
    
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                        rotation_range = 40,
                                        width_shift_range = 0.2,
                                        height_shift_range = 0.2,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True)
    
    test_datagen = ImageDataGenerator(rescale = 1./255)
    
    valid_datagen = ImageDataGenerator(rescale = 1./255)

    train_generator = train_datagen.flow_from_directory(
        base_dir,
        target_size = (150,150),
        batch_size = 40,
        class_mode = 'categorical')
    
    valid_generator = test_datagen.flow_from_directory(
        test_base_dir,
        target_size = (150,150),
        batch_size = 40,
        class_mode = 'categorical')
        
    history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 10,
    validation_data = valid_generator,
    validation_steps = 40,
    callbacks = [checkpoint, earlystopping])
    

C:\Users\choij\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=15.
  warnings.warn(("The least populated class in y has only %d"


Found 4277 images belonging to 88 classes.
Found 2154 images belonging to 88 classes.


<ipython-input-48-5a6faa8afd4c>:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
100/100 [==============================] - 158s 2s/step - loss: 1.7534 - accuracy: 0.7043 - val_loss: 4.8242 - val_accuracy: 0.0675
Epoch 2/10
100/100 [==============================] - 146s 1s/step - loss: 1.0871 - accuracy: 0.8224 - val_loss: 5.3524 - val_accuracy: 0.0669
Epoch 3/10
100/100 [==============================] - 149s 1s/step - loss: 0.9096 - accuracy: 0.8359 - val_loss: 6.2336 - val_accuracy: 0.0694
Epoch 4/10
100/100 [==============================] - 154s 2s/step - loss: 0.8550 - accuracy: 0.8351 - val_loss: 6.6606 - val_accuracy: 0.0594
Epoch 5/10
100/100 [==============================] - 147s 1s/step - loss: 0.7997 - accuracy: 0.8401 - val_loss: 7.4565 - val_accuracy: 0.0512
Epoch 6/10
100/100 [==============================] - 147s 1s/step - loss: 0.7912 - accuracy: 0.8394 - val_loss: 10.4868 - val_accuracy: 0.0581
Epoch 7/10
100/100 [==============================] - 149s 1s/step - loss: 0.7595 - accuracy: 0.8414 - val_loss: 9.2524 - val_accuracy: 0.052

IndexError: index 3724 is out of bounds for axis 0 with size 3724